In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
import pickle
import gc

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_curve
from scipy.fftpack import dct

In [ ]:
#Load features
X = np.load('../SavedFeatures/X_mel_spec.npy',  allow_pickle=True)
y = np.load('../SavedFeatures/y_mel_spec.npy',  allow_pickle=True)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Đổi trường dữ liệu 
X_train = X_train.reshape(X_train.shape[0], -1)  # to 2D
X_val = X_val.reshape(X_val.shape[0], -1)  # to 2D

del X, y  
gc.collect()

#### Super Vector Machine -------------------------

In [10]:
svm_model = make_pipeline(StandardScaler(), SVC(kernel='rbf', probability=True))
svm_model.fit(X_train, y_train)


In [ ]:
svm_val_predictions = svm_model.predict(X_val)
accuracy = accuracy_score(y_val, svm_val_predictions)
f1_class_0 = f1_score(y_val, svm_val_predictions, pos_label=0)  
f1_class_1 = f1_score(y_val, svm_val_predictions, pos_label=1)  
f1_avg = (f1_class_0 + f1_class_1) / 2

# Tính EER
y_val_scores = svm_model.decision_function(X_val) 
fpr, tpr, thresholds = roc_curve(y_val, y_val_scores)
frr = 1 - tpr
eer_index = np.nanargmin(np.abs(fpr - frr))
eer = fpr[eer_index] 
eer_threshold = thresholds[eer_index]



print("====================SVM - MS====================")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score  {f1_avg:.4f}")
print("\nClassification Report:\n", classification_report(y_val, svm_val_predictions))
print(f"Equal Error Rate (EER): {eer:.4f}")
print(f"EER Threshold: {eer_threshold:.4f}")


====================SVM - MS====================
Accuracy: 0.9161
F1-Score  0.9093

Classification Report:
               precision    recall  f1-score   support

           0       0.88      1.00      0.93      1904
           1       0.99      0.80      0.88      1280

    accuracy                           0.92      3184
   macro avg       0.94      0.90      0.91      3184
weighted avg       0.92      0.92      0.91      3184

Equal Error Rate (EER): 0.0620
EER Threshold: -0.5501


In [ ]:
with open("../SavedModel/MS_svm_dfd.pkl", 'rb') as file:
    svm_model = pickle.load(file)

In [ ]:
del svm_model, X_train, X_val, y_train, y_val
gc.collect()